In [ ]:
## IMPORTS
import pandas as pd 
import requests
import praw
import time
import pprint
import logging
import json
import os
import re
from prawcore.exceptions import RequestException, ResponseException, ServerError
from newspaper import Article
from newsapi import NewsApiClient
from bs4 import BeautifulSoup
from requests.exceptions import RequestException 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import pandas as pd
import os
import re
import glob 

import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')




import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
## INIT APIs 

newsKey = '992b1247b5b448e3935baac5af4a841c' #coryFlynn
newsAPI = NewsApiClient(api_key=newsKey)

#searchQuery=''"AI art" OR "AI Image" OR "AI Artists" OR "AI-generated art" OR "AI generated images" or "AI generation" OR "generative AI art" OR "algorithmic art" -OLED -Pendlebury -Pypi.org -The-next-web -byteDance -samsung',

#searchQuery='"art" OR "painting" OR "drawings" OR "illustration" OR "illustrator" "graphic design" OR "animation" OR "fine arts" OR "mural" OR "creative work" OR "art exhibition" OR "museum" OR "gallery" -"AI art" -"AI-generated" -"AI Images" -"machine learning art" -OLED -The-next-web -byteDance -samsung',

logging.basicConfig(
    level=logging.DEBUG,  #see all logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    force=True  # rest config
)

#INIT reddit PRAW API 
reddit = praw.Reddit(
    client_id='JBh-NQdNIpzU5_QWija3LQ', 
    client_secret='AjBxNAXhL4hPvdWf6xYUpA4sg9ATLg', 
    user_agent='script:top_posts_scraper:v1.0 (by u/bingabanggg)',
)

# subredditName = 'aiArt'  #name of subreddits to scrape
# subreddit = reddit.subreddit(subredditName)

#          - https://www.reddit.com/r/aiArt/ (PRO AI ART) 
#           - https://www.reddit.com/r/Art/ (ANTI AI ART) 

In [ ]:
##HELPER FUNCTIONS

def cleanText(text):
    """
    run this on any text to remove punctuation, whitespace, and nonalphabetic characters
    """
    if not text:
        return ""
    
    #remove cruft and punctuation, extra spaces
    text = re.sub(r'[,.;@#?!&$\-\']+', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'\"', ' ', text, flags=re.IGNORECASE)
    text = re.sub(' +', ' ', text, flags=re.IGNORECASE)

    # remove nonalphabetical (keep spaces)
    text = re.sub(r'[^a-zA-Z ]', ' ', text, flags=re.VERBOSE)
    text = ' '.join(text.split())
    return text

def getDateFromIso(isoDate):
    """
    given an ISO will return date 
    """
    if not isoDate:
        return ""
    return isoDate.split("T")[0]


In [ ]:
# def bs4ArticleScrape(url, newsApiExcerpt=None):
#     """
#     given the article desc from newsAPI (which is cut around200 chars because of free api tier)
#     this funct checks if the content scraped from bs4 in a <p> section matches the init part of the excerpt 
#     (so as to confirm it scraped the right content) and then returns the page content as a str. 
#     """
#     fullContent = "" #empty str to save the page content to 

#     #using try blcok so that errors wont break the system 
#     try:
#         response = requests.get(url, timeout=10)            #send get request
#         time.sleep(1)                                       # time delay to be nice to the server
#         soup = BeautifulSoup(response.text, 'html.parser')  # take raw page content and use html parser
#         paragraphs = soup.find_all('p')                     #grab all text in paragraphs, returns as list of <p>
#         fullContent = ' '.join([p.get_text() for p in paragraphs]).strip()  #grabs only the text from lists of p
        
#         #compare scraped para to the article desc from newsAPI to see if we grabbed the 'article' 
#         if newsApiExcerpt:                                  #check if there is a desc that exists
#             cleanedExcerpt = cleanText(newsApiExcerpt)      #use clean text function to clean both texts
#             cleanedFull = cleanText(fullContent)            
#             if cleanedExcerpt and cleanedExcerpt not in cleanedFull: #check 1) if clean text is not empty and 2) that cleaned exerpt not in cleanfull:
#                 print("Warning: The scraped content does not appear to include the expected NewsAPI excerpt.")
#                 #if article desc and scraped content dont match, printed response will let me know 
#     except Exception as e:
#         print(f"Error fetching full article from {url}: {e}")
#     return fullContent

In [ ]:


# test URL
testUrl = "https://www.androidpolice.com/ai-art-in-project-zomboids-update-sparks-community-outrage/"


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

try:
    response = requests.get(testUrl, headers=headers, timeout=10)
    response.raise_for_status()  # yell when theres a bad responses
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract main article content (i skimmed, and sites use <article> or <p>)
    paragraphs = soup.find_all("p")
    full_text = "\n".join([p.get_text() for p in paragraphs])

    print("\n--- Extracted Full Article ---\n")
    print(full_text[:1000])  # Print first 1000 characters

except requests.exceptions.RequestException as e:
    print(f"Error fetching article: {e}")


In [ ]:
##fetch articles and get them as JSON

def fetchNewsArticles(searchQuery, pageSize=20, sortBy="relevancy", testMode=False):
    """
    request to newsAPI and save as JSON file with all data. 
    testmode limits the call to one page 
    """

    allArticles = []
    maxPages = 1 if testMode else 5  # Test mode limits to 1 page for quick testing
    fromDate = (datetime.datetime.now() - datetime.timedelta(days=30)).strftime('%Y-%m-%d')  # Get max allowed date

    for page in range(1, maxPages + 1):  #get max amnt of pages (API thing)
        try:
            #tell what page we're on, make api request
            print(f"Fetching page {page}...")

            response = newsAPI.get_everything(
                q=searchQuery,
                language="en",
                sort_by=sortBy,
                page=page,
                page_size=pageSize,
                from_param=fromDate,
            )
        
            if "articles" in response: #return count of articles retreived per page
                articlesCount = len(response["articles"])
                print(f"Page {page} returned {articlesCount} articles.")

                # Extract relevant fields
                for article in response["articles"]:
                        #each of these has a value for if it doesnt exist - this helps flagging articles for data cleaning review later 
                    structured_article = {
                        "source": article["source"]["name"] if article.get("source") else "Unknown", #conditional: if source exists, get name, else "unknown"
                        "author": article.get("author", "Unknown"),
                        "title": article.get("title", "No Title"),
                        "description": article.get("description", "No Description"),
                        "content": article.get("content", "No Content Available"),
                        "url": article.get("url", "No URL"),
                        "publishedAt": article.get("publishedAt", "No Date"),
                    }
                    allArticles.append(structured_article)

            else:
                print(f"Page {page} returned no articles.")

        except Exception as e:
            print(f"Error fetching page {page}: {e}") 
            break

        # Ratelimit safety
        time.sleep(2)

    #timestamp of file (for file management craziness)
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    jsonFileName = f"newsAPI_{timestamp}.json"

    #save and return as JSON
    with open(jsonFileName, "w", encoding="utf-8") as jsonFile:
        json.dump(allArticles, jsonFile, indent=4, ensure_ascii=False)

    print(f"Articles saved to {jsonFileName}")

    return jsonFileName


In [ ]:
# #EXPERIMENTING WITH QUERIES IN FUNCTION 'testing mode' 
# #LETS ME SEE WHAT TERMS WORK THE BEST TO GET MOST RELEVANT ARTICLES 

# #AI ART SEARCH 1 - DIDNT WORK, RETURNED ARTICLES RELATING TO 'AI' ONLY. 
# searchQuery = "AI art"
# language = "en"
# pageSize = 100
# sortBy = "relevancy"
# fromDate = "2024-12-27"
# language = 'en'


# #ATTEMPT 2 
# #WORKS - THERES SOME PRODUCTS THAT OFFER AI GEN ART 
# #AND SO PRODUCT ARTICLES ARE FILTERED IN.. THEYRE STILL RELEVANT THO. 
# searchQuery = '"AI art" OR "AI-generated art" OR "artificial intelligence art" OR "AI artwork"'
# language = "en"
# pageSize = 100
# sortBy = "relevancy"
# fromDate = "2024-12-27"

# searchQuery = (
#     '"AI art" OR "AI-generated art" OR "artificial intelligence art" OR "AI artwork" '
#     'OR "generative art" OR "machine learning art" OR "AI creativity" '
#     'OR "AI artist" OR "AI-created art" OR "neural network art" OR "algorithmic art"'
# )


# searchQuery = (
#     '"Midjourney" OR "Stable Diffusion" OR "DALL-E" OR "DALL-E 2" OR "RunwayML" OR "Leonardo AI" '
#     'OR "Deep Dream" OR "DeepDream" OR "Artbreeder" OR "Dream by Wombo" OR "Wombo Dream" '
#     'OR "Fotor AI" OR "NightCafe AI" OR "StarryAI" OR "Pixray" OR "DeepAI Image Generator" '
#     'OR "Dreamlike Art" OR "DeepArt" OR "CF Spark" OR "Playground AI" '
#     'OR "Deep Dream Generator" OR "RunDiffusion" OR "Mage AI" OR "Imagine AI" '
#     'OR "InstantID" OR "GauGAN" OR "Synthesys AI" OR "Unreal Diffusion"'
# )

'''

"art" OR "painting" OR "drawings" OR "illustration" OR 
"illustrator" "graphic design" OR "animation" OR "fine arts" OR "mural" OR 
"creative work" OR "art exhibition" OR "museum" OR "gallery" -"AI art" 
-"AI-generated" -"AI Images" -"machine learning art" -OLED -The-next-web -byteDance -samsung

'''

In [ ]:

#ATTEMPT 2 
#WORKS - THERES SOME PRODUCTS THAT OFFER AI GEN ART 
#AND SO PRODUCT ARTICLES ARE FILTERED IN.. THEYRE STILL RELEVANT THO. 
searchQuery = '"AI art" OR "AI-generated art" OR "artificial intelligence art" OR "AI artwork"'
language = "en"
pageSize = 100
sortBy = "relevancy"
fromDate = "2024-12-27"

#this runs the function in testmode to experiment without making rate limit an issue
testArticles = fetchNewsArticles(
    searchQuery=searchQuery,
    pageSize=10,  # Smaller page size for testing
    testMode=True  # Enable test mode
)


In [ ]:
##GET ARTICLES FOR ART 
###searchQuery='"art" OR "painting" OR "drawings" OR "illustration" OR "illustrator" "graphic design" OR "animation" OR "fine arts" OR "mural" OR "creative work" OR "art exhibition" OR "museum" OR "gallery" -"AI art" -"AI-generated" -"AI Images" -"machine learning art" -"OLED" -"The-next-web" -"byteDance" -"samsung"'



#fetchNewsArticles('"art" OR "painting" OR "drawings" OR "illustration" 
#                   OR "illustrator" OR "graphic design" OR "animation" OR "fine arts" 
#                   OR "mural" OR "creative work" OR "art exhibition" OR "museum" OR "gallery" 
#                   -"AI art" -"AI-generated" -"AI Images" -"machine learning art" -"OLED" -"The-next-web" -"byteDance" -"samsung"')


In [ ]:
def loadArticlesFromJSON(jsonFile):
    """
    Reads output JSON from fetchNewsArticles articles and converts it to a df
    """
    with open(jsonFile, "r", encoding="utf-8") as file:
        articles = json.load(file)

    df = pd.DataFrame(articles)
    
    # make sure all columns are as they should be (otherwise add them with None values to be flagged later)
    expectedColumns = ["source", "author", "title", "description", "content", "url", "publishedAt"]
    for col in expectedColumns:
        if col not in df:
            df[col] = None
    
        if col not in df:
            df[col] = None
            
    # lambda to each element in source column of DF - checks if element is dict with key 'name' - if yes, return name if not, return unknown. 
    # if isinstance(source, dict) and "name" in source:
    #     return source["name"]
    # else:
    #     return "Unknown"
    df["source"] = df["source"].apply(lambda x: x["name"] if isinstance(x, dict) and "name" in x else "Unknown")

    return df

In [ ]:
articles = loadArticlesFromJSON('newsAPI_2025-02-12_21-20-39.json')
articles.head(4)

In [ ]:
#set keywords from the original query, these will be used to evaluate if the content scraped is truly relevant.
###searchQuery='"art" OR "painting" OR "drawings" OR "illustration" OR "illustrator" "graphic design" OR "animation" OR "fine arts" OR "mural" OR "creative work" OR "art exhibition" OR "museum" OR "gallery" -"AI art" -"AI-generated" -"AI Images" -"machine learning art" -"OLED" -"The-next-web" -"byteDance" -"samsung"'
Keywords = [
    "art", "painting", "drawings", "illustration", "illustrator",
    "graphic design", "animation", "fine arts", "mural", "creative work",
    "art exhibition", "museum", "gallery"
]

### apply this to df to check if keywords appear in the text(exact matches only)
def checkExactKeyword(text):
    """
    Check if any Art-related keyword appears as an exact phrase in the text.
    """
    if isinstance(text, str):
        return any(f" {kw} " in f" {text} " for kw in Keywords)
    return False

In [ ]:
import pandas as pd

# Load the CSV file
filePath = "ALL DATA/ART_articles.csv"
df = pd.read_csv(filePath)

# Select two articles for comparison (adjust the indices if needed)
article_1 = df.iloc[0]  # First article
article_2 = df.iloc[1]  # Second article

# Print a comparison of the fields
print("Comparison of Article Fields:\n")

print(f"Title 1: {article_1['title']}")
print(f"**Content** 1:\n{article_1['content']}\n")
print(f"**FullArticleContent** 1:\n{article_1['FullArticleContent']}\n")



print(f"Title 2: {article_2['title']}\n")
print(f"**Content**:\n{article_2['content']}\n")
print(f"**FullArticleContent** 2:\n{article_2['FullArticleContent']}\n")


In [ ]:
###this checks for keywords and will ask approval for articles that pop up with not enough//missing content 
def filterAndProcessArticles(df):
    """
    goes article by article to check for keywords actually IN the article content/body. 
    then flags for bad-data and has you check them one by one (usually only a dozen will need checking.)
    """
    #drop any rows missing content 
    df.dropna(subset=['title', 'description', 'content', 'url'], inplace=True)

    #keyword checks on all text columns
    df["RelevantTitle"] = df["title"].apply(checkExactKeyword)
    df["RelevantContent"] = df["content"].apply(checkExactKeyword)
    df["RelevantDescription"] = df["description"].apply(checkExactKeyword)

    #articles that dont match any keywords are flagged (new columns)
    df["KeywordMatch"] = df[["RelevantTitle", "RelevantContent", "RelevantDescription"]].any(axis=1)

    #all articles from keyword process that got flagged are grabbed for manual review
    #all articles that have less than 50 chars are also flagged for review
    df["ContentLength"] = df["content"].str.len().fillna(0) + df["description"].str.len().fillna(0)
    df["NeedsManualReview"] = (df["ContentLength"] < 50) | (~df["KeywordMatch"])

    return df

In [ ]:
###  fetch the full article content 
def fetchFullArticle(url):
    """
    given url will use newspaper3k first then bs4 to try and get full article text. 
    """

    try:
        article = Article(url)
        article.download()
        article.parse()
        
        # check if the len grabbed is significant len
        if len(article.text) > 100:
            return article.text.strip()
        
        #if newspaper3k falls thru then use bs4
        else:
            print(f"Newspaper3k extracted too little content for {url}. Trying BS4 fallback...")
    except Exception as e:
        print(f"Newspaper3k failed for {url}: {e}. Trying BS4 fallback...")

    # using bs4 instead 

    try:
        response = requests.get(url, headers=headers, timeout=10)

        #skip if blocked 
        if response.status_code in [403, 429] or "consent" in response.url:
            print(f" Skipping (Blocked/Consent Page): {url}")
            return None

        response.raise_for_status()

        # find p tags and extract text. 
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all("p")
        fullText = "\n".join([p.get_text() for p in paragraphs]).strip()

        return fullText if len(fullText) > 100 else None  # minimum length for results so that bad articles automatically get cut

    except Exception as e:
        print(f"BS4 also failed for {url}: {e}")
        return None

In [ ]:
### clean save to DF
def saveProcessedData(df):
    approvedRows = df[df["NeedsManualReview"] == False]
    rejectedRows = df[df["NeedsManualReview"] == True]

    approvedRows.to_csv(cleanFileName, index=False)
    rejectedRows.to_csv(rejectedFileName, index=False)

    print(f"Clean articles saved to {cleanFileName}")
    print(f"Rejected articles saved to {rejectedFileName}")


In [ ]:
#master function runs all on given json file. 
def processAllArticles(jsonFile):
    """
    Master function that loads, processes, and saves articles from a NewsAPI JSON file.
    Ensures every article gets full-length content.
    """
    print("Loading JSON data...")
    df = loadArticlesFromJSON(jsonFile)
    
    print("Filtering and processing articles...")
    df = filterAndProcessArticles(df)

    print("Fetch full article content for given articles")
    df["FullArticleContent"] = df["url"].apply(fetchFullArticle) 

    print("saving processed data...")
    saveProcessedData(df)

    print("file complete.")

In [ ]:
# cleanFileName="clean_art_articles.csv"
# rejectedFileName="rejected_art_articles.csv"

processAllArticles("newsAPI_2025-02-16_21-38-46.json")


In [ ]:
## EXPERIMENTING WITH REDDIT

In [ ]:
# subreddit_name = 'aiArt'  #name of subreddits to scrape
# subreddit = reddit.subreddit(subreddit_name)

#          - https://www.reddit.com/r/aiArt/ (PRO AI ART) 
#           - https://www.reddit.com/r/Art/ (ANTI AI ART) 

In [ ]:
def isBotComment(commentText):
    botPhrases = ["i am a bot", "automated", "bot", "auto-mod"]
    return any(phrase in commentText for phrase in botPhrases)

def handleRateLimit(waitSeconds=60):
    logging.warning(f"Rate limit reached. Waiting for {waitSeconds} seconds...")
    time.sleep(waitSeconds)


#this will let me immediately save posts//incrimentally as I scrape reddit.
def savePost(postData, filePath):
    """append to jsonl"""
    try:
        with open(filePath, 'a', encoding='utf-8') as f:
            f.write(json.dumps(postData, ensure_ascii=False) + '\n')
    except IOError as e:
        logging.error(f"Failed to write to file {filePath}: {e}")

#COUNT LINES IN JSONL FILE
def countJsonlLines(filePath):
    """returns the number of lines in the json file if it exists (filename issues lol)"""
    if os.path.exists(filePath):
        with open(filePath, 'r', encoding='utf-8') as f:
            return sum(1 for _ in f)
    return 0

def loadExistingData(filePath):
    """makes sure json file exists before loading"""
    if not os.path.exists(filePath):
        return []
    with open(filePath, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]



In [ ]:
##THIS IS ONLY FOR AI RELATED FILES
#TO PROCESS THEM FOR MY USE NOW 

'''
bc news api has the 30 day limit I wanted ot make the most use out of even the super
 messy early data files I had retrieved, especially for the URLs since I plan to use Bs4 to get the full articles anyways. '''

def scrapeSubreddit(subredditName):
    """
    goes through posts and sorts by ai related and nonAI related posts in a given subreddit. 
    grabs from hot , controverisal, and top and stops when it has 150 total posts related to AI 
    """
    logging.info(f"Starting scrape for subreddit: {subredditName}")

    #create two CSV files, named for the source subreddit, where posts will be filtered.
    #one file contains all posts with keywords relating to AI in them
    #other file is for safekeeping of all posts that dont contain keywords. 
    AIPostFile = f"{subredditName}_AIposts.json"
    nonAIPostFile = f"{subredditName}_nonAIPosts.json"

    #parameters for scrape sample
    maxAIPosts = 150  
    topLimit = 70  #ALL TIME
    controversialLimit = 60  #ALL-TIME
    hotLimit = 20  #RECENT

    #save the post ids of everything already scraped so there arent duplicates
    processedPostIDs = set()

    #retries for larger loop 
    maxRetries = 3  
    retryCount = 0  

    try:
        subreddit = reddit.subreddit(subredditName)

        totalCollectedAI = countJsonlLines(AIPostFile) #count progress towards 150 AIrelated posts by using the file theyre saved to
        
        #grab posts from TOP 
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {topLimit} posts from TOP...")
            processPosts(subreddit.top(time_filter="all", limit=topLimit), AIPostFile, nonAIPostFile, processedPostIDs)

        #grab posts from Controversial
        totalCollectedAI = len(loadExistingData(AIPostFile))
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {controversialLimit} posts from CONTROVERSIAL...")
            processPosts(subreddit.controversial(time_filter="all", limit=controversialLimit), AIPostFile, nonAIPostFile, processedPostIDs)

        #grab posts from HOT
        totalCollectedAI = len(loadExistingData(AIPostFile))
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {hotLimit} posts from HOT...")
            processPosts(subreddit.hot(limit=hotLimit), AIPostFile, nonAIPostFile, processedPostIDs)

        #check total collected posts related to AI, then backfill to 150 with posts from TOP
        while totalCollectedAI < maxAIPosts:
            neededPosts = maxAIPosts - totalCollectedAI
            logging.info(f"Filling gap with {neededPosts} additional TOP posts...")
            processPosts(subreddit.top(time_filter="all", limit=neededPosts), AIPostFile, nonAIPostFile, processedPostIDs)
            totalCollectedAI = len(loadExistingData(AIPostFile))

            #continue retrying until len 150. 
            if totalCollectedAI == len(loadExistingData(AIPostFile)):
                retryCount += 1  
                logging.warning(f"No new AI posts found. Retry attempt {retryCount}/{maxRetries}.")
            else:
                retryCount = 0  
            
            #in case of small subreddits with few related posts; stops after 3 tries
            if retryCount >= maxRetries:
                logging.warning("Max retries reached. Not enough AI posts in this subreddit. Stopping scrape.")
                break 
    except (RequestException, ResponseException, ServerError) as e:
        logging.error(f"Reddit API error: {e}. Retrying...")
        handleRateLimit()
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

    logging.info(f"Finished scraping {subredditName}. Total AI posts collected: {totalCollectedAI}")

In [ ]:
## THIS IS HOW I SORTED POSTS 

def processPosts(posts, AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords):
    """
    takes in a list of keywords to search for in a post's text content, and uses regex to be sure that word does or does not appear in the post. 
    searches the comments then sorts by content in comments 
    """
    
    #regex pattern for finding AI-related keywords. uses word boundaries (\b) to ensure full word matching
    keywordPattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in aiKeywords) + r')\b', re.IGNORECASE)

    for post in posts:
        try:
            if post.id in processedPostIDs:
                continue
            processedPostIDs.add(post.id)

            postText = f"{post.title} {post.selftext}".strip().lower()

            # Initialize post data
            postData = {
                "postId": post.id,
                "title": post.title,
                "selftext": post.selftext,
                "score": post.score,
                "numComments": post.num_comments,
                "subreddit": post.subreddit.display_name,
                "createdUtc": post.created_utc,
                "url": post.url,
                "comments": []
            }

            # Fetch comments and remove bot-generated ones
            post.comments.replace_more(limit=0)
            allComments = post.comments.list()

            # Separate top and controversial comments
            topComments = [c for c in allComments[:25] if not isBotComment(c.body.lower())]
            controversialComments = [c for c in allComments[-10:] if not isBotComment(c.body.lower())]

            # Filter comments: Must have at least 3 words & score > 1
            filteredComments = [
                c for c in (topComments + controversialComments)
                if len(c.body.split()) >= 3 and c.score > 1
            ][:30]  # Keep max 30 comments

            for comment in filteredComments:
                postData["comments"].append({
                    "commentId": comment.id,
                    "body": comment.body,
                    "score": comment.score,
                    "createdUtc": comment.created_utc
                })

            # Skip posts with no text and insufficient comments
            if not post.selftext.strip() and len(postData["comments"]) < 5:
                logging.info(f"Skipping post {post.id}: No text and insufficient comments.")
                continue  

            #determine post related-ness 
            # Check if post or comments contain any AI-related keywords (with whole-word matching)
            isAiRelated = bool(keywordPattern.search(postText)) or any(
                keywordPattern.search(comment["body"]) for comment in postData["comments"]
            )

            # Save to appropriate file
            if isAiRelated:
                savePost(postData, AIPostFile)
                logging.info(f"AI-related post saved: {post.id}")
            else:
                savePost(postData, nonAIPostFile)
                logging.info(f"Non-AI-related post saved: {post.id}")
            
            time.sleep(2)  # Rate-limit API requests
    
        except (RequestException, ResponseException, ServerError) as e:
            logging.error(f"API Error: {e}. Retrying after delay.")
            time.sleep(60)  # Prevent API block 
            continue
        except Exception as e:
            logging.error(f"Unexpected error processing post {post.id}: {e}")
            continue


In [ ]:
##CREATE KEYWORDS USING VOCABULARY FROM MAIN SCRAPE OF AI WARS 

def loadJsonl(filePath):
    """Loads JSONL data into a list."""
    data = []
    with open(filePath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove usernames
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation
    return text.strip()


In [ ]:
# AIWars_AIposts.csv	 - AI 	
# Art_ArtPosts.csv      - ART 
# ArtistHate_Aiposts.csv  -AI 	
# DigitalPainting_AIDisc.csv    -AI 
# ART_articles.csv		   -ART
# DigitalPainting_ArtPosts.csv -ART
# ArtBuisness_AIDisc.csv	-ART 
# Illustration_AIDisc.csv   -AI 
# ArtBusiness_posts.csv		-ART
# Illustration_ArtPosts.csv -ART
# Art_AIDisc.csv		 - AI 	
# aiARt_AIposts.csv - AI 

In [ ]:
# Configuration
#KEYWORDS = ["ai art", "generative", "midjourney", "stable diffusion", "stealing", "originality", "automation"]

# keywords = [
#     "AI art", "generative art", "AI-generated art", "Midjourney", "Stable Diffusion", 
#     "DALL-E", "DALL-E 2", "DeepArt", "Artbreeder", "prompt engineering", "diffusion", "used AI", "using Ai",
#     "text-to-image", "AI ethics", "AI copyright", "AI plagiarism", "AI originality", "human artists", "made by human", "not ai", 
#     "AI stealing", "generated", "proAi", "antiAI", " noAI", " no Ai ",
#     "software", " AI", " GAI ", "GenAI", "AIgen", "AIart", "machine learning", "prompt", " generative ai"]

aiKeywords = [
    "painting", "art", "drawing", "artist", "quality", "art style", "style", "sketches", "doodle", "artist", "AI art", "generative art", "AI-generated art", "Midjourney", "Stable Diffusion", 
    "DALL-E", "DALL-E 2", "DeepArt", "Artbreeder", "prompt engineering", "diffusion", "used AI", "using Ai",
    "text-to-image", "AI ethics", "AI copyright", "AI plagiarism", "AI originality", "human artists", "made by human", "not ai", 
    "AI stealing", "generated", "proAi", "antiAI", " noAI", " no Ai ",
    "software", " AI", " GAI ", "GenAI", "AIgen", "AIart", "machine learning", "prompt", " generative ai"]



#didnt feel like making it match exactly so added spaces to two letter keywords 
botPhrases = [
    "i am a bot",
    "this is an automated reminder from the mod team",
    "this action was performed automatically",
    "please contact the moderators"
]

In [ ]:
##REVISED SCRAPE FUNCTION

def scrapeSubreddit(subredditName):
    """
    Scrapes posts from a specified subreddit and classifies them as AI-related or non-AI-related.
    
    - Creates two JSON files to store AI-related and non-AI-related posts.
    - Defines post scrape parameters (limits for different categories).
    - Iteratively fetches posts from 'top', 'controversial', and 'hot' categories.
    - Ensures no duplicate posts are processed.
    - Keeps track of collected AI posts to reach the target limit.
    - Implements a retry mechanism to handle cases where not enough AI posts are found.
    """
    logging.info(f"Starting scrape for subreddit: {subredditName}")

    #create two CSV files, named for the source subreddit, where posts will be filtered.
    #one file contains all posts with keywords relating to AI in them
    #other file is for safekeeping of all posts that dont contain keywords. 
    AIPostFile = f"{subredditName}_AIposts.json"
    nonAIPostFile = f"{subredditName}_nonAIPosts.json"

    #parameters for scrape sample
    maxAIPosts = 150  
    topLimit = 70  #ALL TIME
    controversialLimit = 60  #ALL-TIME
    hotLimit = 20  #RECENT
    # ################ TEST ################
    # topLimit = 100  #ALL TIME
    # controversialLimit = 30  #ALL-TIME
    # hotLimit = 20  #RECENT
    # #######################################

    #save the post ids of everything already scraped so there arent duplicates
    processedPostIDs = set()

    #retries for larger loop 
    maxRetries = 3  
    retryCount = 0  

    try:
        subreddit = reddit.subreddit(subredditName)

        totalCollectedAI = countJsonlLines(AIPostFile) #count progress towards 150 AIrelated posts by using the file theyre saved to
        
        #grab posts from TOP 
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {topLimit} posts from TOP...")
            processPosts(subreddit.top(time_filter="all", limit=topLimit), AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords)

        #grab posts from Controversial
        totalCollectedAI = len(loadExistingData(AIPostFile))
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {controversialLimit} posts from CONTROVERSIAL...")
            processPosts(subreddit.controversial(time_filter="all", limit=controversialLimit), AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords)

        #grab posts from HOT
        totalCollectedAI = len(loadExistingData(AIPostFile))
        if totalCollectedAI < maxAIPosts:
            logging.info(f"Fetching {hotLimit} posts from HOT...")
            processPosts(subreddit.hot(limit=hotLimit), AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords)

        #check total collected posts related to AI, then backfill to 150 with posts from TOP
        while totalCollectedAI < maxAIPosts:
            neededPosts = maxAIPosts - totalCollectedAI
            logging.info(f"Filling gap with {neededPosts} additional TOP posts...")
            processPosts(subreddit.top(time_filter="all", limit=neededPosts), AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords)
            totalCollectedAI = len(loadExistingData(AIPostFile))

            #continue retrying until len 150. 
            if totalCollectedAI == len(loadExistingData(AIPostFile)):
                retryCount += 1  
                logging.warning(f"No new AI posts found. Retry attempt {retryCount}/{maxRetries}.")
            else:
                retryCount = 0  
            
            #in case of small subreddits with few related posts; stops after 3 tries
            if retryCount >= maxRetries:
                logging.warning("Max retries reached. Not enough AI posts in this subreddit. Stopping scrape.")
                break 
    except (RequestException, ResponseException, ServerError) as e:
        logging.error(f"Reddit API error: {e}. Retrying...")
        handleRateLimit()
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

    logging.info(f"Finished scraping {subredditName}. Total AI posts collected: {totalCollectedAI}")

In [ ]:
import re
import logging
import time
from requests.exceptions import RequestException
from prawcore.exceptions import ResponseException, ServerError

def processPosts(posts, AIPostFile, nonAIPostFile, processedPostIDs, aiKeywords):
    """
    Processes Reddit posts and categorizes them as AI-related or non-AI-related using precise keyword matching.
    
    - Uses regex to avoid substring issues (e.g., "paint" triggering "AI").
    - Filters bot-generated and low-quality comments.
    - Accepts a list of AI-related keywords for flexibility.
    """
    
    #regex pattern for finding AI-related keywords. uses word boundaries (\b) to ensure full word matching
    keywordPattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in aiKeywords) + r')\b', re.IGNORECASE)

    for post in posts:
        try:
            if post.id in processedPostIDs:
                continue
            processedPostIDs.add(post.id)

            postText = f"{post.title} {post.selftext}".strip().lower()

            # Initialize post data
            postData = {
                "postId": post.id,
                "title": post.title,
                "selftext": post.selftext,
                "score": post.score,
                "numComments": post.num_comments,
                "subreddit": post.subreddit.display_name,
                "createdUtc": post.created_utc,
                "url": post.url,
                "comments": []
            }

            # Fetch comments and remove bot-generated ones
            post.comments.replace_more(limit=0)
            allComments = post.comments.list()

            # Separate top and controversial comments
            topComments = [c for c in allComments[:25] if not isBotComment(c.body.lower())]
            controversialComments = [c for c in allComments[-10:] if not isBotComment(c.body.lower())]

            # Filter comments: Must have at least 3 words & score > 3
            filteredComments = [
                c for c in (topComments + controversialComments)
                if len(c.body.split()) >= 3 and c.score > 3
            ][:15]  # Keep max 15 comments

            for comment in filteredComments:
                postData["comments"].append({
                    "commentId": comment.id,
                    "body": comment.body,
                    "score": comment.score,
                    "createdUtc": comment.created_utc
                })

            # Skip posts with no text and insufficient comments
            if not post.selftext.strip() and len(postData["comments"]) < 5:
                logging.info(f"Skipping post {post.id}: No text and insufficient comments.")
                continue  

            #determine post related-ness 
            # Check if post or comments contain any AI-related keywords (with whole-word matching)
            isAiRelated = bool(keywordPattern.search(postText)) or any(
                keywordPattern.search(comment["body"]) for comment in postData["comments"]
            )
            

            # Save to appropriate file
            if isAiRelated:
                savePost(postData, AIPostFile)
                logging.info(f"AI-related post saved: {post.id}")
            else:
                savePost(postData, nonAIPostFile)
                logging.info(f"Non-AI-related post saved: {post.id}")
            
            time.sleep(2)  # Rate-limit API requests
    
        except (RequestException, ResponseException, ServerError) as e:
            logging.error(f"API Error: {e}. Retrying after delay.")
            time.sleep(60)  # Prevent API block 
            continue
        except Exception as e:
            logging.error(f"Unexpected error processing post {post.id}: {e}")
            continue


In [ ]:
scrapeSubreddit("aiArt")
scrapeSubreddit("AIWars")
scrapeSubreddit("ArtistHate")


In [ ]:
# Art-related keywords
artKeywords = [ "painting", "art", "drawing", "artist", "quality", "art style", "style", "sketches", "doodle", "artist","painting", "illustration", "drawing", "fine art", "traditional", "oil painting",
               "sculpture", "gallery", "commission", "watercolor", "sketching", "canvas", "pastels"]


aiKeywords = [
    "AI art", "generative art", "AI-generated art", "Midjourney", "Stable Diffusion", 
    "DALL-E", "DALL-E 2", "DeepArt", "Artbreeder", "prompt engineering", "diffusion", "used AI", "using Ai",
    "text-to-image", "AI ethics", "AI copyright", "AI plagiarism", "AI originality", "human artists", "made by human", "not ai", 
    "AI stealing", "generated", "proAi", "antiAI", " noAI", " no Ai ", "AI", "artificial intelligence", "machine learning", "generative", "Stable Diffusion",
    "Midjourney", "DALL-E", "AI-generated", "AI art", "prompt", "deep learning", "GPT",
    "software", " AI", " GAI ", "GenAI", "AIgen", "AIart", "machine learning", "prompt", " generative ai"]

# regex patterns
artPattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in artKeywords) + r')\b', re.IGNORECASE)
aiPattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in aiKeywords) + r')\b', re.IGNORECASE)

# check top 15 comments for AI discussion
def scrapeArtSubreddit(subredditName, limit=100):
    logging.info(f"Starting scrape for Art subreddit: {subredditName}")

    # Output files
    artPostsFile = f"{subredditName}_ArtPosts.json"
    aiDiscussedFile = f"{subredditName}_AI_Discussions.json"

    #Track processed posts
    processedPostIDs = set()

    try:
        subreddit = reddit.subreddit(subredditName)

        posts = list(subreddit.hot(limit=limit)) + \
                list(subreddit.top(time_filter="all", limit=limit)) + \
                list(subreddit.controversial(time_filter="all", limit=limit)) + \
                list(subreddit.new(limit=limit))

        collectedAI = 0
        for post in posts:
            if post.id in processedPostIDs:
                continue  # Skip duplicate posts
            processedPostIDs.add(post.id)

            postText = f"{post.title} {post.selftext}".strip().lower()

            # Ensure the post is Art-related
            if not artPattern.search(postText):
                continue  # Skip non-art-related posts

            #grab top 15 comments
            post.comments.replace_more(limit=0)
            allComments = post.comments.list()[:15]

            #Fetch comments and remove bot-generated ones
            filteredComments = [c.body for c in allComments if not isBotComment(c.body)]

            #this is the actual search happening
            isAiDiscussed = any(aiPattern.search(comment) for comment in filteredComments) 

            
            postData = {
                "postId": post.id,
                "title": post.title,
                "selftext": post.selftext,
                "subreddit": subredditName,
                "category": "Art" if not isAiDiscussed else "AI_Discussion", #label for if AI is discussed or not 
                "num_comments": post.num_comments,
                "score": post.score,
                "url": post.url,
                "top_comments": filteredComments
            }

            if isAiDiscussed:
                collectedAI += 1
                savePost(postData, aiDiscussedFile)
                logging.info(f"AI-related discussion found in comments. Saved post: {post.id}")
            else:
                savePost(postData, artPostsFile)
                logging.info(f"Saved non-AI Art post: {post.id}")

        logging.info(f"Finished scraping {subredditName}. AI-related discussions found in {collectedAI} posts.")

    except (RequestException, ResponseException, ServerError) as e:
        logging.error(f"Reddit API error: {e}. Retrying...")
        handleRateLimit()
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

# Function to save posts to JSON
def savePost(postData, jsonFile):
    with open(jsonFile, 'a') as f:
        json.dump(postData, f)
        f.write('\n')


In [ ]:
# Function to clean text
def clean_text(text, keywords=KEYWORDS_TO_REMOVE):
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase for consistent processing
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove non-ASCII characters (emojis, special symbols)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove new lines
    text = text.replace("\n", " ").replace("\r", " ")
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove specific keywords
    for keyword in keywords:
        text = re.sub(r'\b' + re.escape(keyword) + r'\b', '', text, flags=re.IGNORECASE)
    
    return text

def processTextData(df, keywords=KEYWORDS_TO_REMOVE):
    # Select relevant text columns
    text_columns = ["title", "description", "content"]
    
    # Apply cleaning function to each text column
    for col in text_columns:
        df[col] = df[col].apply(lambda x: clean_text(x, keywords))
    
    # Keep only LABEL, text data columns, and source
    cleaned_df = df[["LABEL"] + text_columns + ["source"]]
    
    return cleaned_df

In [ ]:
#new one 

def cleanText(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'[^\x00-\x7F]+', '', text) 
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

# Define file mapping for AI and ART categories
ai_files = [
    "AIWars_AIposts.csv", "ArtistHate_Aiposts.csv", "DigitalPainting_AIDisc.csv",
    "Illustration_AIDisc.csv", "Art_AIDisc.csv", "aiARt_AIposts.csv"
]

art_files = [
    "Art_ArtPosts.csv", "DigitalPainting_ArtPosts.csv",
    "ArtBuisness_AIDisc.csv", "ArtBusiness_posts.csv", "Illustration_ArtPosts.csv"
]

columns = ["LABEL", "source", "author", "title", "description", "content", "url", "publishedAt"]

In [ ]:

# Define keywords to remove (can be changed outside the function)
KEYWORDS_TO_REMOVE = ["AI art", "generative art", "AI-generated art", "Midjourney", "Stable Diffusion", 
    "DALL-E", "DALL-E 2", "DeepArt", "Artbreeder", "prompt engineering", "diffusion", "used AI", "using Ai",
    "text-to-image", "AI ethics", "AI copyright", "AI plagiarism", "AI originality", "human artists", "made by human", "not ai", 
    "AI stealing", "generated", "proAi", "antiAI", " noAI", " no Ai ", "AI", "artificial intelligence", "machine learning", "generative", "Stable Diffusion",
    "Midjourney", "DALL-E", "AI-generated", "AI art", "prompt", "deep learning", "GPT",
    "software", " AI", " GAI ", "GenAI", "AIgen", "AIart", "machine learning", "prompt", " generative ai", "painting", "art", "drawing", "artist", "quality", "art style", "style", "sketches", "doodle", "artist","painting", "illustration", "drawing", "fine art", "traditional", "oil painting",
               "sculpture", "gallery", "commission", "watercolor", "sketching", "canvas", "pastels"]


In [ ]:
#merge the csvs as a df

def process_csvs(file_list, label):
    all_data = []
    for file in file_list:
        df = pd.read_csv(file, usecols=columns)
        all_data.append(df)
    
    merged_df = pd.concat(all_data, ignore_index=True)
    
    #save the pre-cleaning version
    unclean_filename = f"uncleaned_text{label}.csv"
    merged_df.to_csv(unclean_filename, quoting=1)
    
    #run cleaning functions on text columns
    for col in ["title", "description", "content"]:
        merged_df[col] = merged_df[col].apply(clean_text)
    
    #save as cleaned
    cleaned_filename = f"{label}_posts.csv"
    merged_df.to_csv(cleaned_filename, quoting=1)
    
    return merged_df

In [ ]:
ai_posts_df = process_csvs(ai_files, "AI")
art_posts_df = process_csvs(art_files, "Art")

In [ ]:
df = pd.read_csv("uncleaned_textAI.csv")
cleanedDF = processTextData(df)
cleanedDF.to_csv("cleaned_NO_KW_AIPosts.csv",)

In [ ]:
#LabelDataDF_AI.dropna(subset=["content"], inplace=True)
len(LabelDataDF_AI)

In [ ]:
len(LabelDataDF_Art)

In [ ]:
LabelDataDF_AI.to_csv("LDF-NKW-AiArt.csv")
print ("Labelled Dataframe: sampled for AIArt.")
LabelDataDF_AI.sample(10)

In [ ]:
LabelDataDF_Art.to_csv("LDF-NKW-Art.csv")
print ("Labelled Dataframe: sampled for ART.")
LabelDataDF_Art.sample(10)

In [ ]:

def removeStopwords(text, stopWords):
    tokens = word_tokenize(text)
    return " ".join([word for word in tokens if word.lower() not in stopWords])

def lemmatizeText(text, lemmatizer):
    tokens = word_tokenize(text)
    return " ".join([lemmatizer.lemmatize(word) for word in tokens])

def tokenizeText(text):
    return word_tokenize(text)

def vectorizeText(text):
    vectorizerCount = CountVectorizer(stop_words='english', max_features=1000)
    countMatrix = vectorizerCount.fit_transform(text)
    dfCount = pd.DataFrame(countMatrix.toarray(), columns=vectorizerCount.get_feature_names_out())

    vectorizerTfidf = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidfMatrix = vectorizerTfidf.fit_transform(text)
    dfTfidf = pd.DataFrame(tfidfMatrix.toarray(), columns=vectorizerTfidf.get_feature_names_out())
    
    return dfCount, dfTfidf

def processTextData(df, prefix):
    stopWords = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    # Ensure "content" column is string type
    df = df.copy()
    df["content"] = df["content"].astype(str)
    
    # 1. Remove stopwords
    dfCleaned = df.copy()
    dfCleaned["content"] = dfCleaned["content"].apply(lambda x: removeStopwords(x, stopWords))
    
    # 2. Lemmatize text
    dfLemmatized = dfCleaned.copy()
    dfLemmatized["content"] = dfLemmatized["content"].apply(lambda x: lemmatizeText(x, lemmatizer))
    
    # 3. Tokenize text
    dfTokenized = dfCleaned.copy()
    dfTokenized["content"] = dfTokenized["content"].apply(tokenizeText)
    
    # 4. Vectorize the text using CountVectorizer and TfidfVectorizer
    dfCount, dfTfidf = vectorizeText(dfCleaned["content"])
    
    # Print sample outputs
    print(f"Lemmatized DF Sample for {prefix}:")
    print(dfLemmatized.head())
    
    print(f"\nTokenized DF Sample for {prefix}:")
    print(dfTokenized.head())
    
    print(f"\nCountVectorizer DF Sample for {prefix}:")
    print(dfCount.head())
    
    print(f"\nTfidfVectorizer DF Sample for {prefix}:")
    print(dfTfidf.head())
    
    # Return as DFs with names
    return {
        f"{prefix}_lemmatized": dfLemmatized,
        f"{prefix}_tokenized": dfTokenized,
        f"{prefix}_count": dfCount,
        f"{prefix}_tfidf": dfTfidf
    }


In [ ]:
processTextData("LDF-NKW-AiArt.csv", "AIArt")

In [ ]:
processTextData("LDF-NKW-Art.csv", "Art")

In [ ]:
print(AIArtDF_tfidf.columns())
print("AI Art TFIDF")

print(AIArtDF_count.columns())
print("AIArt Count Vectorized")

In [ ]:
AIArtDF_count.to_csv("AIPosts_CountVectorized.csv")
AIArtDF_lemmatized.to_csv("AIPosts_lemmatized.csv")
AIArtDF_tfidf.to_csv("AIPosts_tfidf.csv")
AIArtDF_tokenized.to_csv("AIPosts_tokenized.csv")

In [ ]:
ArtDF_count.to_csv("ArtPosts_CountVectorized.csv")
ArtDF_lemmatized.to_csv("ArtPosts_lemmatized.csv")
ArtDF_tfidf.to_csv("ArtPosts_tfidf.csv")
ArtDF_tokenized.to_csv("ArtPosts_tokenized.csv")

In [ ]:
word_frequencies = ArtDF_count.sum(axis=0).to_dict()

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_frequencies)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("WordCloud for ArtDF CountVectorized")
plt.show()


In [ ]:
word_frequencies = AIArtDF_count.sum(axis=0).to_dict()
countVectCloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_frequencies)

plt.figure(figsize=(10, 5))
plt.imshow(countVectCloud, interpolation="bilinear")
plt.axis("off")
plt.title("WordCloud for AIArtDF CountVectorized")
plt.show()

In [ ]:
ArtDF_count.head(40)

In [ ]:
tempDF = AIArtDF_count.drop(columns=["like"], inplace=False)

In [ ]:
word_frequencies = tempDF.sum(axis=0).to_dict()
countVectCloud2 = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_frequencies)


plt.figure(figsize=(10, 5))
plt.imshow(countVectCloud2, interpolation="bilinear")
plt.axis("off")
plt.title("WordCloud for AIArtDF CountVectorized")
plt.show()

In [ ]:
uncleaned_datasets = {
    "AIARTDF_count.csv": "AI",
    "AIArtDF_tfidf.csv": "ART",
    "AIArtDF_lemmatized.csv": "ART",
}

cleaned_datasets = {
    "cleaned_NO_KW_AIPosts.csv": "AI",
    "cleaned_NO_KW_ArtPOSTS.csv": "ART",
    "LDF-NKW-AiArt.csv": "AI",
    "LDF-NKW-Art.csv": "ART",
}
vectorized_datasets = {
    "AIPosts_lemmatized.csv": "AI",
    "AIPosts_tokenized.csv": "AI",
    "ArtPosts_lemmatized.csv": "ART",
    "ArtPosts_tokenized.csv": "ART",
}

#colors based on sorted categories
category_colormaps = {"AI": "Reds", "ART": "Blues"}
source_colors = {"NEWS API": "lightgray", "REDDIT": "darkgray"}

In [ ]:
#function for wordclouds
def generate_wordcloud(filePath, title, category):
    df = pd.read_csv(filePath)
    text = " ".join(df.astype(str).values.flatten())
    wordcloud = WordCloud(width=800, height=400, background_color=source_colors.get(title.split(" - ")[0], "white"), colormap=category_colormaps.get(category, "viridis")).generate(text)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title, fontsize=14, fontweight="bold")
    plt.show()

In [ ]:
for file, category in cleaned_datasets.items():
    filePath = os.path.join(file)
    generate_wordcloud(filePath, f"{file} - [UNCLEANED]", category)

In [ ]:
for file, category in cleaned_datasets.items():
    filePath = os.path.join(file)
    generate_wordcloud(filePath, f"{file} - [UNCLEANED]", category)

In [ ]:

for file, category in vectorized_datasets.items():
 filePath = os.path.join(file)
    generate_wordcloud(filePath, f"{file} - [CLEANED]", category)


In [ ]:
for file, category in vectorized_datasets.items():
    filePath = os.path.join(os.getcwd(), file)  # Uses current working directory
    generate_wordcloud(filePath, f"{file}", category)